In [ ]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import pymysql
import nest_asyncio


nest_asyncio.apply()

# Conexión a la base de datos
db = pymysql.connect(
    host="localhost",
    user="root",
    password="43340837",
    database="libreria",
    port=3306
)
cursor = db.cursor()

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Función para procesar cada categoría
async def procesar_categoria(categoria, categoria_url, session):
    print(f"Procesando categoría: {categoria}")
    page = 1

    while True:
        print(f"Procesando página {page} de la categoría '{categoria}'...")
        url = f"{categoria_url}?page={page}"
        try:
            async with session.get(url) as response:
                if response.status != 200:
                    print(f"Error al acceder a la página {page}. Código de estado: {response.status}")
                    break

                soup = BeautifulSoup(await response.text(), "html.parser")
                product_elements = soup.select("div.box-producto")
                if not product_elements:
                    print(f"No se encontraron más productos en la página {page}.")
                    break

                for product in product_elements:
                    try:
                        # Extracción de datos del producto
                        product_name = product.select_one("h3.nombre").text.strip() if product.select_one("h3.nombre") else "N/A"
                        product_author = product.select_one("div.autor").text.strip() if product.select_one("div.autor") else "N/A"
                        product_price_now = product.select_one("p.precio-ahora strong").text.strip() if product.select_one("p.precio-ahora strong") else "0"
                        product_price_before = product.select_one("p.precio-antes del").text.strip() if product.select_one("p.precio-antes del") else "0"
                        discount = product.select_one("div.descuento-v2").text.strip() if product.select_one("div.descuento-v2") else "N/A"
                        product_url = product.select_one("a").get("href", "").strip() if product.select_one("a") else "N/A"
                        additional_details = product.select_one("div.autor.color-dark-gray.metas").text.strip() if product.select_one("div.autor.color-dark-gray.metas") else "N/A"
                        stock = product.select_one("div.stock.color-green").text.strip() if product.select_one("div.stock.color-green") else "N/A"
                        image_url = product.select_one("img").get("data-src", "").strip() if product.select_one("img") else "N/A"

                        # Manejo de precios
                        try:
                           
                            product_price_now = product_price_now.replace(",", "").replace("$", "").strip()
                            product_price_now = float(product_price_now)
                        except ValueError:
                            product_price_now = 0.0

                        try:
                            
                            product_price_before = product_price_before.replace(",", "").replace("$", "").strip()
                            product_price_before = float(product_price_before)
                        except ValueError:
                            product_price_before = 0.0

                        if not discount or discount == "N/A":
                            product_price_before = product_price_now

                        # Insertar datos en la base de datos
                        cursor.execute(
                            """
                            INSERT INTO productos (categoria, nombre, autor, precio_actual, precio_anterior, descuento, url_producto, detalles_adicionales, unidades_disponibles, imagen_url)
                            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                            """,
                            (
                                categoria,
                                product_name,
                                product_author,
                                product_price_now,
                                product_price_before,
                                discount,
                                product_url,
                                additional_details,
                                stock,
                                image_url,
                            )
                        )
                        db.commit()
                    except Exception as e:
                        print(f"Error al insertar un producto: {e}")
        except Exception as e:
            print(f"Error al realizar la solicitud para {url}: {e}")
            break

        page += 1
        await asyncio.sleep(3) 

async def main():
    base_url = "https://www.buscalibre.com.co/"
    categories = []

    # Obtener las categorías
    async with aiohttp.ClientSession(headers=headers) as session:
        response = await session.get(base_url)
        if response.status != 200:
            print(f"Error al acceder a la página principal. Código de estado: {response.status}")
            return

        soup = BeautifulSoup(await response.text(), "html.parser")
        category_elements = soup.select("li.category-li a")
        for element in category_elements:
            try:
                name = element.find("span").text.strip()
                category_url = element["href"].strip()
                categories.append((name, category_url))
            except Exception as e:
                print(f"Error al procesar una categoría: {e}")

        # Procesar todas las categorías en paralelo
        tasks = [procesar_categoria(name, url, session) for name, url in categories]
        await asyncio.gather(*tasks)

await main()

cursor.close()
db.close()

print("Proceso completado.")
